# 使用 API 快速搭建你的第一个 AI 应用
> [HW3：以 AI 搭建自己的應用](https://colab.research.google.com/drive/15jh4v_TBPsTyIBhi0Fz46gEkjvhzGaBR?usp=sharing)中文镜像版
> 
> 指导文章：[02. 简单入门：通过 API 与 Gradio 构建 AI 应用](https://github.com/Hoper-J/LLM-Guide-and-Demos/blob/master/Guide/02.%20简单入门：通过%20API%20与%20Gradio%20构建%20AI%20应用.md)

目标：了解如何通过使用 API 和 Prompt 来构建自己的语言模型应用。

在线链接：[Colab](https://colab.research.google.com/drive/1nz89ATcl5l900bso6-xErEo1laT-rmPE?usp=share_link)



# 安装库

In [ ]:
!pip install openai
!pip install gradio
!pip install 'numpy<2.0'

# 导入与设置

In [ ]:
import os
import json
from typing import List, Dict, Tuple

import openai
import gradio as gr

In [ ]:
# TODO: 设置你的 OPENAI API 密钥，这里假设 DashScope API 被配置在了 OPENAI_API_KEY 环境变量中
OPENAI_API_KEY = ""
# 不填写则默认使用环境变量
if not OPENAI_API_KEY:
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# 初始化 OpenAI 客户端，使用阿里云 DashScope API
client = openai.OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # 阿里云的 API 地址
)

# 检查 API 设置是否正确
try:
    response = client.chat.completions.create(
        model="qwen-turbo",  # 使用通义千问-Turbo 大模型，可以替换为 Deepseek 系列：deepseek-v3 / deepseek-r1
        messages=[{'role': 'user', 'content': "测试"}],
        max_tokens=1,
    )
    print("API 设置成功！！")
except Exception as e:
    print(f"API 可能有问题，请检查：{e}")

## 第1部分：文章摘要（单轮对话应用）

在此任务中，你需要将你的聊天机器人变为一个**摘要器**。它的工作是当用户输入一篇文章时，能够为用户总结该文章的内容。

你需要完成以下步骤：

1. 设计一个用于生成摘要的提示词，并填写在 **prompt_for_summarization** 中。
2. **点击运行按钮**， 这将弹出一个可交互的界面。
3. 你可以找到一篇文章或使用当前的示例文章：《从百草园到三味书屋》，并将其填写在标记为“文章”的输入框中。
4. 点击“发送”按钮生成文章的摘要。（你可以使用“温度”滑块来控制输出的创造性，温度越高，输出越具创造性）。
5. 如果你**想更改提示词**，可以停止单元格，返回到TODO部分进行更改，然后再次运行。
7. 在你获得满意的结果后，点击“导出”按钮保存结果。文件列表中将出现一个名为 **part1.json** 的文件。

注意：

- **如果你再次点击“导出”按钮，之前的结果将被覆盖。**
- **即使使用相同的提示词，输出的结果可能仍然不同。**

------

在运行此单元格之前，请确保已运行 **安装包** 和 **导入与设置**。

**记得在进行下一步前停止此单元格。**


In [ ]:
# TODO: 修改提示词以满足你的摘要需求
PROMPT_FOR_SUMMARIZATION = "请将以下文章概括成几句话。"

def reset():
    """
    清空对话记录

    返回:
        List: 空的对话记录列表
    """
    return []

def interact_summarization(prompt, article, temp=1.0):
    """
    调用模型生成摘要。

    参数:
        prompt (str): 用于摘要的提示词
        article (str): 需要摘要的文章内容
        temp (float): 模型温度，控制输出创造性（默认 1.0）

    返回:
        List[Tuple[str, str]]: 对话记录，包含输入文本与模型输出
    """
    # 合成请求文本
    input_text = f"{prompt}\n{article}"
    
    response = client.chat.completions.create(
        model="qwen-turbo",  # 使用通义千问-Turbo大模型
        messages=[{'role': 'user', 'content': input_text}],
        temperature=temp,
    )
    return [(input_text, response.choices[0].message.content)]

def export_summarization(chatbot, article):
    """
    导出摘要任务的对话记录和文章内容到 JSON 文件。

    参数:
        chatbot (List[Tuple[str, str]]): 模型对话记录
        article (str): 文章内容
    """
    target = {"chatbot": chatbot, "article": article}
    with open("files/part1.json", "w", encoding="utf-8") as file:
        json.dump(target, file, ensure_ascii=False, indent=4)

# 构建 Gradio UI 界面
with gr.Blocks() as demo:
    gr.Markdown("# 第1部分：摘要\n填写任何你喜欢的文章，让聊天机器人为你总结！")
    chatbot = gr.Chatbot()
    prompt_textbox = gr.Textbox(label="提示词", value=PROMPT_FOR_SUMMARIZATION, visible=False)
    article_textbox = gr.Textbox(label="文章", interactive=True, value="填充")
    
    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出，温度越高，响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")
    
    with gr.Row():
        send_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")
    
    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")
    
    # 绑定按钮与回调函数
    send_button.click(interact_summarization,
                      inputs=[prompt_textbox, article_textbox, temperature_slider],
                      outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_summarization, inputs=[chatbot, article_textbox])

# 启动 Gradio 应用
demo.launch(debug=True)

### 检查并打印你的结果

此部分用于检查你的“part1.json”文件是否包含所有正确的上下文，或者用于查看我们提供的示例文件。

你需要：

1. 确保文件列表中有你保存的 **part1.json** 文件。
2. 点击运行按钮。它将显示一个**冻结的** Gradio 界面，重现你的摘要结果。

---

在运行此单元格之前，请确保已运行 **安装库** 和 **导入与设置**。

**记得在进行下一步前停止此单元格。**

In [ ]:
# 加载对话记录的 JSON 文件
with open("files/part1.json", "r") as f:
    context = json.load(f)

chatbot = context['chatbot']  # 获取对话记录
article = context['article']  # 获取原始文章
summarization = chatbot[0][-1]  # 获取摘要结果

# 构建 Gradio UI 界面
with gr.Blocks() as demo:
    gr.Markdown("# 第1部分：摘要\n你可以查看文章和摘要！")
    chatbot = gr.Chatbot(value=context['chatbot'])  # 加载对话历史
    article_textbox = gr.Textbox(label="文章", interactive=False, value=context['article'])  # 显示原始文章

    # 构建展示摘要和原文的部分
    with gr.Column():
        gr.Markdown("# 只是一个检查")
        gr.Textbox(label="文章", value=article, show_copy_button=True)  # 显示并允许复制原文
        gr.Textbox(label="摘要", value=summarization, show_copy_button=True)  # 显示并允许复制摘要

# 启动 Gradio 应用
demo.launch(debug=True)

## 第2部分：角色扮演（多轮对话应用）

在此任务中，你需要将聊天机器人设定为**角色扮演模式**。你应该为它指定一个角色，然后通过提示让它进入该角色的状态。

你需要完成以下步骤：

1. 想出一个你希望聊天机器人扮演的**角色**，以及一个使聊天机器人进入该角色的提示词。在 **character_for_chatbot** 中填写角色，在 **prompt_for_roleplay** 中填写提示词。
2. **点击运行按钮，界面将弹出一个可交互的界面。**
3. **与聊天机器人进行** 2 轮 **互动**。在标为“输入”的框中输入你想说的话，然后点击“发送”按钮。（你可以使用“温度”滑块来控制输出的创造性。）
4. 如果你**想更改提示词或角色**，可以停止单元格，返回TODO重新设置，然后重新运行单元格。
5. 在你获得满意的结果后，点击“导出”按钮保存结果。文件列表中将出现一个名为 **part2.json** 的文件。

注意：

- **如果你再次点击“导出”按钮，之前的结果将被覆盖。**
- **即使使用相同的提示词，输出的结果可能仍然不同。**

------

在运行此单元格之前，请确保已运行 **安装包** 和 **导入与设置**。

**记得在进行下一步前停止此单元格。**



In [ ]:
# TODO: 修改以下变量以定义角色和角色提示词
CHARACTER_FOR_CHATBOT = "面试官"  # 机器人扮演的角色，注意，真正起作用的实际是提示词，因为并没有预设 system 角色
PROMPT_FOR_ROLEPLAY = "我需要你面试我有关AI的知识，仅提出问题"  # 指定角色提示词

# 清除对话的函数
def reset():
    """
    清空对话记录。

    返回:
        List: 空的对话记录列表
    """
    return []

# 调用模型生成对话的函数
def interact_roleplay(chatbot, user_input, temp=1.0):
    """
    处理角色扮演多轮对话，调用模型生成回复。

    参数:
        chatbot (List[Tuple[str, str]]): 对话历史记录（用户与模型回复）
        user_input (str): 当前用户输入
        temp (float): 模型温度参数（默认 1.0）

    返回:
        List[Tuple[str, str]]: 更新后的对话记录
    """
    try:
        # 构建包含历史对话的消息列表
        messages = []
        for input_text, response_text in chatbot:
            messages.append({'role': 'user', 'content': input_text})
            messages.append({'role': 'assistant', 'content': response_text})
        
        # 添加当前用户输入
        messages.append({'role': 'user', 'content': user_input})
        
        # 调用 API 获取回复
        response = client.chat.completions.create(
            model="qwen-turbo",
            messages=messages,
            temperature=temp,
        )
        chatbot.append((user_input, response.choices[0].message.content))

    except Exception as e:
        print(f"发生错误：{e}")
        chatbot.append((user_input, f"抱歉，发生了错误：{e}"))
        
    return chatbot

def export_roleplay(chatbot, description):
    """
    导出角色扮演对话记录及任务描述到 JSON 文件。

    参数:
        chatbot (List[Tuple[str, str]]): 对话记录
        description (str): 任务描述
    """
    target = {"chatbot": chatbot, "description": description}
    with open("files/part2.json", "w", encoding="utf-8") as file:
        json.dump(target, file, ensure_ascii=False, indent=4)

# 进行第一次对话：设定角色提示
first_dialogue = interact_roleplay([], PROMPT_FOR_ROLEPLAY)

# 构建 Gradio UI 界面
with gr.Blocks() as demo:
    gr.Markdown("# 第2部分：角色扮演\n与聊天机器人进行角色扮演互动！")
    chatbot = gr.Chatbot(value=first_dialogue)
    description_textbox = gr.Textbox(label="机器人扮演的角色", interactive=False, value=CHARACTER_FOR_CHATBOT)
    input_textbox = gr.Textbox(label="输入", value="")
    
    with gr.Column():
        gr.Markdown("# 温度调节\n温度控制聊天机器人的响应创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")
    
    with gr.Row():
        send_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")
    
    with gr.Column():
        gr.Markdown("# 保存结果\n点击导出按钮保存对话记录。")
        export_button = gr.Button(value="导出")
        
    # 绑定按钮与回调函数
    send_button.click(interact_roleplay, inputs=[chatbot, input_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_roleplay, inputs=[chatbot, description_textbox])
    
# 启动 Gradio 应用
demo.launch(debug=True)

### 检查并打印你的结果

此部分用于检查你的 "part2.json" 文件是否包含所有正确的上下文信息，或者用于查看我们提供的示例文件。

你需要：

1. 确保文件列表中有你保存的 **part2.json** 文件。
2. 点击运行按钮。它将显示一个**冻结的** Gradio 界面，重现你的摘要结果。

------

在运行此单元格之前，请确保已运行 **安装库** 和 **导入与设置**。

**记得在进行下一步前停止此单元格。**


In [ ]:
# 加载对话记录的 JSON 文件
with open("files/part2.json", "r") as f:
    context = json.load(f)

# 遍历对话记录并正确存储
chatbot = context['chatbot']
role = context['description']  # 机器人扮演的角色
dialogue = ""  # 对话的完整记录
for i, (user, bot) in enumerate(chatbot):
    if i != 0:
        dialogue += f"用户: {user}\n"  # 用户输入
    dialogue += f"机器人: {bot}\n"  # 机器人回复

# 构建 Gradio UI 界面
with gr.Blocks() as demo:
    gr.Markdown(f"# 第2部分：角色扮演\n聊天机器人和你玩了一个角色扮演游戏，查看结果吧！")
    chatbot = gr.Chatbot(value=context['chatbot'])  # 加载之前的对话记录
    description_textbox = gr.Textbox(label="机器人扮演的角色", interactive=False, value=context['description'])  # 显示角色
    with gr.Column():
        gr.Markdown("# 只是一个检查")
        gr.Textbox(label="角色", value=role, show_copy_button=True)  # 显示并允许复制角色信息
        gr.Textbox(label="对话", value=dialogue, show_copy_button=True)  # 显示并允许复制完整对话

# 启动 Gradio 应用
demo.launch(debug=True)

## 第3部分：定制化任务（多轮对话应用）

在本任务中，你需要提示聊天机器人执行某项特定任务。你应该首先设计一个你希望聊天机器人执行的任务，然后为它提供合适的提示词以引导其完成该任务。

你需要完成以下步骤：

1. 设计一个任务，并根据任务写一个提示词。将任务描述填写在 **chatbot_task** 中，并将提示词填写在 **prompt_for_task** 中。
2. **点击运行按钮**，与聊天机器人进行互动。在 "输入" 框中输入你的内容，然后点击 "发送" 按钮。（你可以使用 "温度" 滑块来控制输出的创造性。）
3. 如果你**想更改提示词或任务名称**，可以停止单元格，返回TODO重新设置，然后重新运行单元格。
4. 在你获得满意的结果后，点击 "导出" 按钮保存结果。文件列表中将出现一个名为 **part3.json** 的文件。

注意：

- **如果你再次点击 "导出" 按钮，之前的结果将被覆盖。**
- **即使使用相同的提示词，输出的结果可能仍然不同。**
- 实际上，这和第2部分是完全一致的，只是将提前预进行的prompt延后到了用户点击发送的时候，使得更像是一个通用的范例，隐藏了角色扮演的属性，但仅仅是隐藏，本质并没有改变。

---

在运行此单元格之前，请确保已运行 **安装包** 和 **导入与设置**。


In [ ]:
# TODO: 修改以下变量以定义任务描述与任务提示词
CHATBOT_TASK = "小学数学老师（输入“开始”）"  # 用于告诉用户聊天机器人可以执行的任务
PROMPT_FOR_TASK = "现在开始，你将扮演一个出小学数学题的老师，当我说开始时提供一个简单的数学题，接收到正确回答后进行下一题，否则给我答案"

def reset():
    """
    清空对话记录。

    返回:
        List: 空的对话记录列表
    """
    return []

def interact_customize(chatbot, prompt, user_input, temp=1.0):
    """
    调用模型处理定制化任务对话。

    参数:
        chatbot (List[Tuple[str, str]]): 历史对话记录
        prompt (str): 指定任务的提示词
        user_input (str): 当前用户输入
        temp (float): 模型温度参数（默认 1.0）

    返回:
        List[Tuple[str, str]]: 更新后的对话记录
    """
    try:
        messages = []
        # 添加任务提示
        messages.append({'role': 'user', 'content': prompt})
        
        # 构建历史对话记录
        for input_text, response_text in chatbot:
            messages.append({'role': 'user', 'content': input_text})
            messages.append({'role': 'assistant', 'content': response_text})

        # 添加当前用户输入
        messages.append({'role': 'user', 'content': user_input})

        response = client.chat.completions.create(
            model="qwen-turbo",
            messages=messages,
            temperature=temp,
            max_tokens=200,  # 修改这个看看
        )

        chatbot.append((user_input, response.choices[0].message.content))

    except Exception as e:
        print(f"发生错误：{e}")
        chatbot.append((user_input, f"抱歉，发生了错误：{e}"))
    
    return chatbot

def export_customized(chatbot, description):
    """
    导出定制化任务对话记录及任务描述到 JSON 文件。

    参数:
        chatbot (List[Tuple[str, str]]): 对话记录
        description (str): 任务描述
    """
    target = {"chatbot": chatbot, "description": description}
    with open("files/part3.json", "w", encoding="utf-8") as file:
        json.dump(target, file, ensure_ascii=False, indent=4)

# 构建 Gradio UI 界面
with gr.Blocks() as demo:
    gr.Markdown("# 第3部分：定制化任务\n聊天机器人可以执行某项任务，试着与它互动吧！")
    chatbot = gr.Chatbot()
    desc_textbox = gr.Textbox(label="任务描述", value=CHATBOT_TASK, interactive=False)
    prompt_textbox = gr.Textbox(label="提示词", value=PROMPT_FOR_TASK, visible=False)
    input_textbox = gr.Textbox(label="输入", value="")
    
    with gr.Column():
        gr.Markdown("# 温度调节\n温度用于控制聊天机器人的输出，温度越高响应越具创造性。")
        temperature_slider = gr.Slider(0.0, 2.0, 1.0, step=0.1, label="温度")
    
    with gr.Row():
        send_button = gr.Button(value="发送")
        reset_button = gr.Button(value="重置")
    
    with gr.Column():
        gr.Markdown("# 保存结果\n当你对结果满意后，点击导出按钮保存结果。")
        export_button = gr.Button(value="导出")
    
    # 绑定按钮与回调函数
    send_button.click(interact_customize, inputs=[chatbot, prompt_textbox, input_textbox, temperature_slider], outputs=[chatbot])
    reset_button.click(reset, outputs=[chatbot])
    export_button.click(export_customized, inputs=[chatbot, desc_textbox])
    
# 启动 Gradio 应用
demo.launch(debug=True)

### 检查并打印你的结果

此部分用于检查你的 "part3.json" 文件是否包含所有正确的上下文信息，或者用于查看我们提供的示例文件。

你需要：

1. 确保文件列表中有你保存的 **part3.json** 文件。
2. 点击运行按钮。它将显示一个**冻结的** Gradio 界面，重现你的摘要结果。

------

在运行此单元格之前，请确保已运行 **安装库** 和 **导入与设置**。

**记得在进行下一步前停止此单元格。**


In [ ]:
# 加载对话记录的 JSON 文件
with open("files/part3.json", "r") as f:
    context = json.load(f)

# 遍历并加载对话记录
chatbot = context['chatbot']
desc = context['description']
dialogue = ""
for user, bot in chatbot:
    dialogue += f"用户: {user}\n"
    dialogue += f"机器人: {bot}\n"

# 构建 Gradio UI 界面
with gr.Blocks() as demo:
    gr.Markdown("# 第3部分：定制化任务\n聊天机器人可以执行特定任务，试着与它互动吧！")
    chatbot = gr.Chatbot(value=context['chatbot'])  # 显示聊天记录
    desc_textbox = gr.Textbox(label="任务描述", value=context['description'], interactive=False)  # 显示任务描述
    
    with gr.Column():
        gr.Markdown("# 只是一个检查")
        gr.Textbox(label="任务描述", value=desc, show_copy_button=True)  # 显示并允许复制任务描述
        gr.Textbox(label="对话记录", value=dialogue, show_copy_button=True)  # 显示并允许复制对话记录

# 启动 Gradio 应用
demo.launch(debug=True)